In [2]:
# Import der notwendigen Bibliotheken
from owslib.wms import WebMapService
import os

## Python Code zur Nutzung des WMS-Servers

In dem folgenden Abschnitt wird der Web Map Service (WMS) von Bremen verwendet, um Karten- und Geodaten zu laden und anzuzeigen. Der Code stellt eine Verbindung zum WMS-Server her, listet verfügbare Layer auf und ruft Informationen zu einem bestimmten Layer ab.

In [3]:
# WMS-URL: Die URL, die den Web Map Service des WMS-Servers angibt.
wms_url = "https://geodienste.bremen.de/wms_dop10_2023?language=ger"

# Verbindung zum Server herstellen
wms = WebMapService(wms_url)

# Verfügbare Layer anzeigen
# Der WMS-Server enthält eine Reihe von Karten-Layern, die wir durchlaufen und ausgeben können.
# In diesem Fall listen wir die Namen der Layer und deren Titel auf.
for layer in wms.contents:
    print(layer, wms[layer].title)

# Informationen über einen bestimmten Layer abrufen
# In diesem Schritt wählen wir einen bestimmten Layer (DOP10_2023_HB) aus und holen uns seine Metadaten.
layer_name = "DOP10_2023_HB"
layer_info = wms[layer_name]
# Bounding Box des Layers: Hier erhalten wir die geographischen Grenzen (Eckkoordinaten) des Layers.
print("Bounding Box Layer:", layer_info.boundingBox)
# Unterstützte CRS (Koordinatenreferenzsysteme): Der WMS-Server kann verschiedene Koordinatensysteme unterstützen. 
# Wir listen die unterstützten CRS für den ausgewählten Layer auf.
print("Supported CRS:", layer_info.crsOptions)

DOP10 2023 Land Bremen DOP10 2023 Land Bremen
DOP10_2023_HB DOP10 2023 Bremen
DOP10_2023_BHV DOP10 2023 Bremerhaven
Bounding Box Layer: (463942.0, 5873090.0, 500928.0, 5898080.0, 'EPSG:25832')
Supported CRS: ['EPSG:25832', 'EPSG:4258', 'EPSG:4326', 'EPSG:3857', 'EPSG:25833', 'CRS:84', 'EPSG:31467', 'EPSG:3044']


## Funktion zum Herunterladen eines Bildes vom WMS-Server

In dieser Funktion wird ein Bild von einem WMS-Server heruntergeladen. Die Funktion verwendet die `getmap`-Methode des WMS-Servers, um das Bild in einem gewünschten Format und einer gewünschten Größe abzurufen und es auf der Festplatte zu speichern.

In [4]:
def download_image(wms, layer, bbox, size, format, filename):
    response = wms.getmap(layers=[layer],   # Wählen des Layers, das heruntergeladen werden soll
                          styles=[''],      # Stil für das Layer (leer bedeutet standardmäßig)
                          srs='EPSG:25832', # Koordinatenreferenzsystem
                          bbox=bbox,        # Geographische Begrenzung (Bounding Box) des Bildes
                          size=size,        # Größe des Bildes in Pixeln (Breite, Höhe)
                          format=format,    # Format des heruntergeladenen Bildes (z.B. 'image/png')
                          transparent=True) # Option für transparente Hintergründe (True oder False)
    
    with open(filename, 'wb') as out:
        out.write(response.read())
    print(f"{filename} erfolgreich heruntergeladen.")

## Python-Code zum Herunterladen von Kartenbildern von einem WMS-Server

Dieser Code zeigt, wie man mit einem WMS-Server Bilder für ein gesamtes Gebiet oder in kleineren Kacheln herunterladen kann. Die Kacheln sind nützlich, um große Karten in handlichere Teile zu unterteilen, die leichter verarbeitet oder angezeigt werden können.

In [14]:
# WMS-Server-URL
url = "https://geodienste.bremen.de/wms_dop10_2023"

# Verbindung zum WMS-Server herstellen
wms = WebMapService(url, version='1.3.0')

# Layer auswählen
layer = 'DOP10_2023_HB'

# Bounding Box definieren (minx, miny, maxx, maxy)
# Diese Werte geben die geographischen Grenzen des Gebiets an, für das Kartenbilder heruntergeladen werden sollen.
bbox = (488608.7545, 5881989.4935, 490215.9319, 5883625.7616)

# Verzeichnis für die Ausgabe erstellen
output_dir = "wms_output"
os.makedirs(output_dir, exist_ok=True)

# Die gewünschte Bildgröße (256x256 Pixel)
tile_size = 1024  # Pixel

# Berechnen der Anzahl der Kacheln, basierend auf der Größe der Bounding Box und der Tilegröße
tile_width = bbox[2] - bbox[0]
tile_height = bbox[3] - bbox[1]

# Berechnen, wie viele Tiles in horizontaler und vertikaler Richtung benötigt werden
num_tiles_x = int(tile_width // tile_size) + (1 if tile_width % tile_size != 0 else 0)
num_tiles_y = int(tile_height // tile_size) + (1 if tile_height % tile_size != 0 else 0)

# Für jede Kachel herunterladen
for i in range(num_tiles_y):
    for j in range(num_tiles_x):
        # Berechnen Sie die Bounding Box für jede Kachel
        tile_bbox = (
            bbox[0] + j * tile_size,
            bbox[1] + i * tile_size,
            bbox[0] + (j + 1) * tile_size,
            bbox[1] + (i + 1) * tile_size
        )

        # Sicherstellen, dass die letzte Kachel in der X- und Y-Richtung nicht über den BBox hinausgeht
        if tile_bbox[2] > bbox[2]:
            tile_bbox = (tile_bbox[0], tile_bbox[1], bbox[2], tile_bbox[3])
        if tile_bbox[3] > bbox[3]:
            tile_bbox = (tile_bbox[0], tile_bbox[1], tile_bbox[2], bbox[3])

        # Den Dateinamen für die Kachel erstellen
        tile_filename = f"tile_{i}_{j}"

        # Downloaden Sie das Bild im gewünschten Format (z.B. TIFF und PNG)
        download_image(wms, layer, tile_bbox, (tile_size, tile_size), 'image/tiff', f"{output_dir}/{tile_filename}_tiff.tiff")
        download_image(wms, layer, tile_bbox, (tile_size, tile_size), 'image/jpeg', f"{output_dir}/{tile_filename}_jpg.jpg")

print("Alle 256x256 Bilder wurden erfolgreich heruntergeladen.")

wms_output/tile_0_0_tiff.tiff erfolgreich heruntergeladen.
wms_output/tile_0_0_jpg.jpg erfolgreich heruntergeladen.
wms_output/tile_0_1_tiff.tiff erfolgreich heruntergeladen.
wms_output/tile_0_1_jpg.jpg erfolgreich heruntergeladen.
wms_output/tile_1_0_tiff.tiff erfolgreich heruntergeladen.
wms_output/tile_1_0_jpg.jpg erfolgreich heruntergeladen.
wms_output/tile_1_1_tiff.tiff erfolgreich heruntergeladen.
wms_output/tile_1_1_jpg.jpg erfolgreich heruntergeladen.
Alle 256x256 Bilder wurden erfolgreich heruntergeladen.


In [21]:
# WMS-Server-URL
url = "https://geodienste.bremen.de/wms_dop10_2023"

# Verbindung zum WMS-Server herstellen
wms = WebMapService(url, version='1.3.0')

# Layer auswählen
layer = 'DOP10_2023_HB'

# Bounding Box definieren (minx, miny, maxx, maxy)
bbox = (488608.7545, 5881989.4935, 490215.9319, 5883625.7616)

# Verzeichnis für die Ausgabe erstellen
output_dir = "wms_output"
os.makedirs(output_dir, exist_ok=True)

# Tile size that matches the model's requirement (256x256 pixels)
tile_size = 256  # This is your model's expected tile size

# Let's define the "zoom" by increasing the resolution (WIDTH and HEIGHT) for smaller tiles
# For example, a higher resolution of 512x512 or 1024x1024 could give better details.
resolution = 8192  # Adjust this to higher values for better detail

# Calculate the new tile count for more zoomed-in tiles
num_tiles_x = int((bbox[2] - bbox[0]) / (resolution / tile_size))  # We divide by the new resolution
num_tiles_y = int((bbox[3] - bbox[1]) / (resolution / tile_size))  # We divide by the new resolution

# Ensure we cover the entire bounding box (round if necessary)
if (resolution / tile_size) % 1 > 0:
    num_tiles_x += 1
if (resolution / tile_size) % 1 > 0:
    num_tiles_y += 1

tile_width = (bbox[2] - bbox[0]) / num_tiles_x
tile_height = (bbox[3] - bbox[1]) / num_tiles_y

# Now, request higher-resolution tiles from the WMS server
for i in range(num_tiles_y):
    for j in range(num_tiles_x):
        tile_bbox = (
            bbox[0] + j * tile_width,
            bbox[1] + i * tile_height,
            bbox[0] + (j + 1) * tile_width,
            bbox[1] + (i + 1) * tile_height
        )

        # Tile filename
        tile_filename = f"tile_{int(tile_bbox[0])}_{int(tile_bbox[1])}"

        # Now, download the tile with higher resolution (we specify WIDTH and HEIGHT in the WMS request)
        download_image(wms, layer, tile_bbox, (resolution, resolution), 'image/jpeg', f"{output_dir}/{tile_filename}_highres.jpg")

print("Alle Bilder wurden erfolgreich heruntergeladen.")


wms_output/tile_488608_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488640_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488673_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488705_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488737_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488769_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488801_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488833_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488865_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488898_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488930_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488962_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_488994_5881989_highres.jpg erfolgreich heruntergeladen.
wms_output/tile_489026_5881989_highres.jpg erfolgreich herunterg

KeyboardInterrupt: 